In [ ]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 0,
  modelName: "qwen-plus",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}
const tongyiChatOptions = {
  alibabaApiKey: process.env.Tongyi_API_KEY,
  temperature: 1.5,
  modelName: "qwen-plus",
};


In [ ]:
import {
  ChatPromptTemplate,
  PromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts"
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi"
import {
  RunnableSequence,
  RunnableWithMessageHistory,
} from "@langchain/core/runnables"
import { StringOutputParser } from "@langchain/core/output_parsers"
import { DynamicStructuredTool, DynamicTool } from "@langchain/core/tools"
import { AgentExecutor, createOpenAIToolsAgent } from "langchain/agents"
import { ChatMessageHistory } from "langchain/stores/message/in_memory"
import z from "zod"


In [ ]:
const mbtiInfo = JSON.parse(await Deno.readTextFile("./mbti-info.json"));
console.log("🚀 ~ mbtiInfo:", mbtiInfo);

const mbtiList = Object.keys(mbtiInfo);

In [ ]:
const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是一个共情能力非常强的心理医生，并且很了解MBTI（迈尔斯-布里格斯性格类型指标)的各种人格类型，你的任务是根据来访者的 MBTI 和问题，给出针对性的情感支持，你的回答要富有感情、有深度和充足的情感支持，引导来访者乐观积极面对问题",
  ],
  [
    "human",
    "用户的 MBTI 类型是{type}, 这个类型的特点是{info}, 他的问题是{question}",
  ],
])

const model = new ChatAlibabaTongyi(tongyiChatOptions)
const mbtiChain = RunnableSequence.from([
  prompt,
  model,
  new StringOutputParser(),
])


In [ ]:
const mbtiTool = new DynamicStructuredTool({
  name: "get-mbti-chat",
  schema: z.object({
    type: z.enum(mbtiList).describe("用户的 MBTI 类型"),
    question: z.string().describe("用户的问题"),
  }),
  func: async ({ type, question }) => {
    const info = mbtiInfo[type]

    const res = await mbtiChain.invoke({ type, question, info })
    return res
  },
  description: "根据用户的问题和 MBTI 类型，回答用户的问题",
})


In [ ]:
const tools = [mbtiTool]

const agentPrompt = await ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是一个用户接待的 agent，通过自然语言询问用户的 MBTI 类型和问题，直到你有足够的信息调用 get-mbti-chat 来回答用户的问题",
  ],
  new MessagesPlaceholder("history_message"),
  ["human", "{input}"],
  new MessagesPlaceholder("agent_scratchpad"),
])

const llm = new ChatAlibabaTongyi(tongyiChatOptions)
const agent = await createOpenAIToolsAgent({
  llm,
  tools,
  prompt: agentPrompt,
})
const agentExecutor = new AgentExecutor({
  agent,
  tools,
})


In [ ]:
const messgaeHistory = new ChatMessageHistory()

const agentWithChatHistory = new RunnableWithMessageHistory({
  runnable: agentExecutor,
  getMessageHistory: () => messgaeHistory,
  inputMessagesKey: "input",
  historyMessageKey: "history_message",
})
